## INSTALL NECESSARY LIBRARIES

In [ ]:
# pip install pennylane pennylane-qiskit 
# pip install networkx gudhi 
# pip install scikit-learn numpy scipy -q
# pip install quri_parts
# pip install torch torch-geometric 
# pip install pylatexenc
# pip install ripser persim 
# pip install seaborn


# IMPORT THE LIBRARIES

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib.patches import Circle, FancyArrowPatch
import warnings
warnings.filterwarnings('ignore')


# LOAD THE DATASETS

In [6]:
print("Loading datasets")

# Load all 5 datasets
datasets = {}
dataset_names = ['MUTAG', 'AIDS', 'PROTEINS', 'NCI1', 'PTC_MR']

for name in dataset_names:
    print(f"Loading {name}", end=" ")
    dataset = TUDataset(root='/tmp/TUDataset', name=name)
    
    # Convert to NetworkX graphs with labels
    graphs = []
    labels = []
    for data in dataset:
        G = to_networkx(data, to_undirected=True)
        # Add node attributes if they exist
        if data.x is not None:
            node_attrs = {i: {'attr': data.x[i].numpy()} for i in range(data.num_nodes)}
            nx.set_node_attributes(G, node_attrs)
        graphs.append(G)
        labels.append(data.y.item())
    
    datasets[name] = {
        'graphs': graphs,
        'labels': np.array(labels),
        'num_classes': len(np.unique(labels)),
        'num_graphs': len(graphs),
        'avg_nodes': np.mean([g.number_of_nodes() for g in graphs]),
        'avg_edges': np.mean([g.number_of_edges() for g in graphs])
    }
    print(f" ({len(graphs)} graphs, {datasets[name]['num_classes']} classes)")

print("\nDataset Statistics:")
print("-" * 70)
for name in dataset_names:
    d = datasets[name]
    print(f"{name:12s} | Graphs: {d['num_graphs']:4d} | Classes: {d['num_classes']:2d} | "
          f"Avg Nodes: {d['avg_nodes']:5.1f} | Avg Edges: {d['avg_edges']:5.1f}")
print("-" * 70)

print("\n All datasets loaded!")

Loading datasets
Loading MUTAG  (188 graphs, 2 classes)
Loading AIDS  (2000 graphs, 2 classes)
Loading PROTEINS  (1113 graphs, 2 classes)
Loading NCI1  (4110 graphs, 2 classes)
Loading PTC_MR  (344 graphs, 2 classes)

Dataset Statistics:
----------------------------------------------------------------------
MUTAG        | Graphs:  188 | Classes:  2 | Avg Nodes:  17.9 | Avg Edges:  19.8
AIDS         | Graphs: 2000 | Classes:  2 | Avg Nodes:  15.7 | Avg Edges:  16.2
PROTEINS     | Graphs: 1113 | Classes:  2 | Avg Nodes:  39.1 | Avg Edges:  72.8
NCI1         | Graphs: 4110 | Classes:  2 | Avg Nodes:  29.9 | Avg Edges:  32.3
PTC_MR       | Graphs:  344 | Classes:  2 | Avg Nodes:  14.3 | Avg Edges:  14.7
----------------------------------------------------------------------

 All datasets loaded!


# FEATURE IMPORTANCE

In [11]:
def extract_spectral_features(G: nx.Graph) -> dict:
    """Extract spectral graph theory features."""
    features = {}
    
    # Laplacian eigenvalues
    try:
        L = nx.laplacian_matrix(G).astype(float)
        n = G.number_of_nodes()
        k = min(10, n-2) if n > 2 else 1
        
        if k > 0:
            eigenvalues = eigsh(L, k=k, which='SM', return_eigenvectors=False)
            eigenvalues = np.sort(eigenvalues)
            
            # Spectral gap (algebraic connectivity)
            features['spectral_gap'] = eigenvalues[1] if len(eigenvalues) > 1 else 0
            
            # Eigenvalue statistics
            features['eigen_mean'] = np.mean(eigenvalues)
            features['eigen_std'] = np.std(eigenvalues)
            features['eigen_max'] = np.max(eigenvalues)
            features['eigen_sum'] = np.sum(eigenvalues)
            
            # Individual eigenvalues (first 5)
            for i in range(min(5, len(eigenvalues))):
                features[f'eigen_{i}'] = eigenvalues[i]
        else:
            # Fallback for small graphs
            features['spectral_gap'] = 0
            features['eigen_mean'] = 0
            features['eigen_std'] = 0
            features['eigen_max'] = 0
            features['eigen_sum'] = 0
            for i in range(5):
                features[f'eigen_{i}'] = 0
                
    except Exception as e:
        # Fallback values
        features['spectral_gap'] = 0
        features['eigen_mean'] = 0
        features['eigen_std'] = 0
        features['eigen_max'] = 0
        features['eigen_sum'] = 0
        for i in range(5):
            features[f'eigen_{i}'] = 0
    
    # Adjacency eigenvalues (for graph energy)
    try:
        A = nx.adjacency_matrix(G).astype(float)
        if A.shape[0] > 2:
            k_adj = min(5, A.shape[0]-2)
            adj_eigenvalues = eigsh(A, k=k_adj, which='LM', return_eigenvectors=False)
            features['graph_energy'] = np.sum(np.abs(adj_eigenvalues))
            features['largest_eigenvalue'] = np.max(np.abs(adj_eigenvalues))
        else:
            features['graph_energy'] = 0
            features['largest_eigenvalue'] = 0
    except:
        features['graph_energy'] = 0
        features['largest_eigenvalue'] = 0
    
    return features


def extract_topological_features(G: nx.Graph) -> dict:
    """Extract topological features (cycles, connectivity, holes)."""
    features = {}
    
    # Basic topology
    features['num_nodes'] = G.number_of_nodes()
    features['num_edges'] = G.number_of_edges()
    features['density'] = nx.density(G)
    
    # Connectivity
    features['is_connected'] = int(nx.is_connected(G))
    features['num_components'] = nx.number_connected_components(G)
    
    # Cycles and loops
    try:
        cycle_basis = nx.cycle_basis(G)
        features['num_cycles'] = len(cycle_basis)
        features['avg_cycle_length'] = np.mean([len(c) for c in cycle_basis]) if cycle_basis else 0
        features['max_cycle_length'] = max([len(c) for c in cycle_basis]) if cycle_basis else 0
        
        # Cycle length distribution
        cycle_lengths = [len(c) for c in cycle_basis]
        for k in [3, 4, 5, 6, 7]:  # Common ring sizes in chemistry
            features[f'cycles_len_{k}'] = sum(1 for l in cycle_lengths if l == k)
    except:
        features['num_cycles'] = 0
        features['avg_cycle_length'] = 0
        features['max_cycle_length'] = 0
        for k in [3, 4, 5, 6, 7]:
            features[f'cycles_len_{k}'] = 0
    
    # Euler characteristic (topological invariant)
    # χ = V - E + F for planar graphs, approximate with cycles
    features['euler_characteristic'] = features['num_nodes'] - features['num_edges'] + features['num_cycles']
    
    # Diameter and radius
    try:
        if nx.is_connected(G):
            features['diameter'] = nx.diameter(G)
            features['radius'] = nx.radius(G)
            features['avg_eccentricity'] = np.mean([nx.eccentricity(G, v) for v in G.nodes()])
        else:
            # Use largest component
            largest_cc = max(nx.connected_components(G), key=len)
            G_sub = G.subgraph(largest_cc)
            features['diameter'] = nx.diameter(G_sub)
            features['radius'] = nx.radius(G_sub)
            features['avg_eccentricity'] = np.mean([nx.eccentricity(G_sub, v) for v in G_sub.nodes()])
    except:
        features['diameter'] = 0
        features['radius'] = 0
        features['avg_eccentricity'] = 0
    
    return features


def extract_chemical_features(G: nx.Graph) -> dict:
    """Extract chemistry-specific features (assuming molecular graphs)."""
    features = {}
    
    # Node attributes 
    if G.number_of_nodes() > 0:
        node_attrs = list(G.nodes(data=True))
        
        # Check if node attributes exist
        has_attrs = 'attr' in node_attrs[0][1] if node_attrs else False
        
        if has_attrs:
            # Atom type diversity (using attribute vectors)
            attr_vectors = [data['attr'] for _, data in node_attrs]
            attr_array = np.array(attr_vectors)
            
            # Feature statistics
            features['attr_mean'] = np.mean(attr_array)
            features['attr_std'] = np.std(attr_array)
            features['attr_max'] = np.max(attr_array)
            features['attr_min'] = np.min(attr_array)
            
            # Diversity (unique attribute patterns)
            unique_attrs = len(set(tuple(row) for row in attr_array))
            features['atom_diversity'] = unique_attrs / len(attr_array) if len(attr_array) > 0 else 0
        else:
            features['attr_mean'] = 0
            features['attr_std'] = 0
            features['attr_max'] = 0
            features['attr_min'] = 0
            features['atom_diversity'] = 0
    
    # Degree distribution (bond counts)
    degrees = [d for n, d in G.degree()]
    if degrees:
        features['avg_degree'] = np.mean(degrees)
        features['std_degree'] = np.std(degrees)
        features['max_degree'] = np.max(degrees)
        features['min_degree'] = np.min(degrees)
        
        # Degree distribution counts
        degree_counts = np.bincount(degrees, minlength=7)
        for i in range(1, 7):  # Degrees 1-6
            features[f'degree_{i}_count'] = degree_counts[i] if i < len(degree_counts) else 0
    else:
        features['avg_degree'] = 0
        features['std_degree'] = 0
        features['max_degree'] = 0
        features['min_degree'] = 0
        for i in range(1, 7):
            features[f'degree_{i}_count'] = 0
    
    # Aromaticity proxy (6-cycles with specific degree patterns)
    try:
        six_cycles = [c for c in nx.cycle_basis(G) if len(c) == 6]
        aromatic_like = 0
        for cycle in six_cycles:
            degrees_in_cycle = [G.degree(n) for n in cycle]
            # Aromatic rings typically have degree 2 or 3 carbons
            if all(2 <= d <= 3 for d in degrees_in_cycle):
                aromatic_like += 1
        features['aromatic_like_rings'] = aromatic_like
    except:
        features['aromatic_like_rings'] = 0
    
    return features


def extract_structural_features(G: nx.Graph) -> dict:
    """Extract structural/motif features (triangles, cliques, paths)."""
    features = {}
    
    # Clustering and transitivity
    features['avg_clustering'] = nx.average_clustering(G)
    features['transitivity'] = nx.transitivity(G)
    
    # Triangle count
    triangles = sum(nx.triangles(G).values()) // 3
    features['num_triangles'] = triangles
    
    # Clique number
    try:
        features['clique_number'] = nx.graph_clique_number(G)
    except:
        features['clique_number'] = 0
    
    # Motif counts (small subgraphs)
    # Count 3-node motifs
    three_node_motifs = 0
    for nodes in nx.enumerate_all_cliques(G):
        if len(nodes) == 3:
            three_node_motifs += 1
        elif len(nodes) > 3:
            break
    features['three_node_motifs'] = three_node_motifs
    
    # Assortativity (degree correlation)
    try:
        features['degree_assortativity'] = nx.degree_assortativity_coefficient(G)
    except:
        features['degree_assortativity'] = 0
    
    # Wiener index (sum of shortest paths)
    try:
        if nx.is_connected(G) and G.number_of_nodes() < 100:  # Limit for computational efficiency
            features['wiener_index'] = nx.wiener_index(G)
        else:
            features['wiener_index'] = 0
    except:
        features['wiener_index'] = 0
    
    # Centrality statistics
    try:
        if G.number_of_nodes() < 500:  # Computational limit
            betweenness = list(nx.betweenness_centrality(G).values())
            features['avg_betweenness'] = np.mean(betweenness)
            features['max_betweenness'] = np.max(betweenness)
            
            closeness = list(nx.closeness_centrality(G).values())
            features['avg_closeness'] = np.mean(closeness)
            features['max_closeness'] = np.max(closeness)
        else:
            features['avg_betweenness'] = 0
            features['max_betweenness'] = 0
            features['avg_closeness'] = 0
            features['max_closeness'] = 0
    except:
        features['avg_betweenness'] = 0
        features['max_betweenness'] = 0
        features['avg_closeness'] = 0
        features['max_closeness'] = 0
    
    return features


def extract_all_features(G: nx.Graph) -> dict:
    """Extract all feature categories."""
    features = {}
    
    features.update(extract_spectral_features(G))
    features.update(extract_topological_features(G))
    features.update(extract_chemical_features(G))
    features.update(extract_structural_features(G))
    
    return features

def compute_feature_importance(graphs: list, labels: np.ndarray, 
                               dataset_name: str = "Dataset") -> pd.DataFrame:
    """
    Compute feature importance using Random Forest.
    
    Args:
        graphs: List of NetworkX graphs
        labels: Target labels
        dataset_name: Name of dataset for display
    
    Returns:
        DataFrame with feature importances
    """
    print(f"\n{'='*70}")
    print(f"FEATURE IMPORTANCE ANALYSIS: {dataset_name}")
    print(f"{'='*70}")
    
    # Extract features for all graphs
    print("Extracting features from all graphs...")
    feature_list = []
    for i, G in enumerate(graphs):
        if (i + 1) % 100 == 0:
            print(f"  Processed {i+1}/{len(graphs)} graphs...")
        features = extract_all_features(G)
        feature_list.append(features)
    
    # Convert to DataFrame
    df_features = pd.DataFrame(feature_list)
    
    # Handle NaN/Inf values
    df_features = df_features.replace([np.inf, -np.inf], np.nan)
    df_features = df_features.fillna(0)
    
    print(f"✓ Extracted {df_features.shape[1]} features from {df_features.shape[0]} graphs")
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_features)
    
    # Train Random Forest for feature importance
    print("Training Random Forest for feature importance...")
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
    rf.fit(X_scaled, labels)
    
    # Get feature importances
    importances = rf.feature_importances_
    feature_names = df_features.columns
    
    # Create importance DataFrame
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances,
        'category': [categorize_feature(f) for f in feature_names]
    }).sort_values('importance', ascending=False)
    
    # Print top features
    print(f"\nTop 20 Most Important Features:")
    print("-" * 70)
    for idx, row in importance_df.head(20).iterrows():
        print(f"{row['feature']:30s} | {row['category']:12s} | {row['importance']:.4f}")
    
    # Category-wise importance
    category_importance = importance_df.groupby('category')['importance'].sum().sort_values(ascending=False)
    print(f"\nImportance by Category:")
    print("-" * 70)
    for category, imp in category_importance.items():
        print(f"{category:15s} | {imp:.4f} ({imp/category_importance.sum()*100:.1f}%)")
    
    return importance_df


def categorize_feature(feature_name: str) -> str:
    """Categorize feature into spectral, topological, chemical, or structural."""
    if any(x in feature_name for x in ['eigen', 'spectral', 'energy', 'laplacian']):
        return 'spectral'
    elif any(x in feature_name for x in ['cycle', 'euler', 'diameter', 'radius', 'component', 'connected', 'eccentricity']):
        return 'topological'
    elif any(x in feature_name for x in ['attr', 'atom', 'aromatic', 'degree']):
        return 'chemical'
    else:
        return 'structural'

def run_feature_importance_analysis(datasets: dict) -> dict:
    """
    Run feature importance analysis on all datasets.
    
    Args:
        datasets: Dictionary from the setup script
    
    Returns:
        Dictionary of importance DataFrames
    """
    importance_results = {}
    
    for dataset_name in ['MUTAG', 'AIDS', 'PROTEINS', 'NCI1', 'PTC_MR']:
        graphs = datasets[dataset_name]['graphs']
        labels = datasets[dataset_name]['labels']
        
        importance_df = compute_feature_importance(graphs, labels, dataset_name)
        importance_results[dataset_name] = importance_df
    
    # Summary recommendations
    print(f"\n{'='*70}")
    print("FEATURE MAP DESIGN RECOMMENDATIONS")
    print(f"{'='*70}")
    
    for dataset_name, importance_df in importance_results.items():
        print(f"\n{dataset_name}:")
        category_imp = importance_df.groupby('category')['importance'].sum().sort_values(ascending=False)
        top_cat = category_imp.index[0]
        print(f"  → Primary focus: {top_cat.upper()} features ({category_imp[top_cat]/category_imp.sum()*100:.1f}%)")
        print(f"  → Top 3 features: {', '.join(importance_df.head(3)['feature'].tolist())}")
    
    return importance_results

importance_results = run_feature_importance_analysis(datasets)


FEATURE IMPORTANCE ANALYSIS: MUTAG
Extracting features from all graphs...
  Processed 100/188 graphs...
✓ Extracted 56 features from 188 graphs
Training Random Forest for feature importance...

Top 20 Most Important Features:
----------------------------------------------------------------------
avg_degree                     | chemical     | 0.1001
num_edges                      | structural   | 0.0951
degree_3_count                 | chemical     | 0.0781
max_betweenness                | structural   | 0.0700
wiener_index                   | structural   | 0.0694
max_closeness                  | structural   | 0.0631
avg_closeness                  | structural   | 0.0609
num_cycles                     | topological  | 0.0573
atom_diversity                 | chemical     | 0.0471
num_nodes                      | structural   | 0.0451
density                        | structural   | 0.0447
avg_eccentricity               | topological  | 0.0417
avg_betweenness                | structura